In [7]:
import pandas as pd
import rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors
import math
import openpyxl



## Data cleaning 
In this chapter we check which of the datsets we have are usefull for our purpose

In [41]:
df_curatedSol = pd.read_csv('../Data/CuratedSol.csv')
df_biogen = pd.read_csv('../Data/Biogen.csv')
df_bioavailability = pd.read_csv('../Data/Bioavailibility.csv')
df_ESOL = pd.read_csv('../Data/ESOL.csv')
df_Water_Octanol = pd.read_csv('../Data/Water_Octanol.csv')
df_Bold_brain_barrier = pd.read_csv('../Data/Blood-brain_barrier_binary.csv')
df_Kinetic_solubility = pd.read_csv('../Data/KineticAqueousSolubility.csv')
df_permeability = pd.read_csv('../Data/Permeability_ph7_4.csv')
df_PAMPA_ph7_4 = pd.read_csv('../Data/PAMPA_ph_7_4.csv')
#df_Pharma_drugs = pd.read_csv('../Data/Pharmaceutical Drugs_Dataset.csv')
df_oral_adsorption = pd.read_csv('../Data/Oral adsorbtion.csv')
df_binary_bioavailibility = pd.read_csv('../Data/Binary_Bioavailibility.csv')
df_OCHEM = pd.read_csv('../Data/OCHEM_Sol.csv')
df_WEI_Bioav = pd.read_csv('../Data/HOB_Bioavailibility_Wei_2022.csv')

In [188]:
#df_curatedSol.head()
#df_bioavailability.head() #995
#df_biogen.head() #3521
#df_ESOL.head() #1128
#df_Water_Octanol.head()#4200
#df_Bold_brain_barrier.head() #2050
#df_Kinetic_solubility.head() #2532
#df_permeability.head() #2530
#df_PAMPA_ph7_4.head() #2532
df_oral_adsorption.head()
#df_binary_bioavailibility.head(20)
#df_water_sol_Ochem.head(100) 


,SMILES,EXTERNALID,N,NAME,ARTICLEID,PUBMEDID,PAGE,TABLE,Oral absorption,UNIT {Oral absorption},...,UNIT {Oral absorption}.1,Species,AssayID,UNIT {AssayID},Molecule ChEMBL ID,UNIT {Molecule ChEMBL ID},Document ChEMBL ID,UNIT {Document ChEMBL ID},Clinical phase,UNIT {Clinical phase}
0,C1(=C(C=C(NC(CCC)=O)C=C1)C(C)=O)OCC(CNC(C)C)O,-,2,Acebutolol,A166,9874698,30,1,90.0,%,...,%,NaN,NaN,-,NaN,-,NaN,-,NaN,-
1,C1(=CC=C(O)C=C1)NC(C)=O,-,3,Acetaminophen,A166,9874698,30,1,80.0,%,...,%,NaN,NaN,-,NaN,-,NaN,-,NaN,-
2,C1(=C(C=CC=C1)OC(C)=O)C(=O)O,-,4,Acetylsalicylic acid,A166,9874698,30,1,100.0,%,...,%,NaN,NaN,-,NaN,-,NaN,-,NaN,-
3,C(C1=NC(=CC=C1)C=CC(O)=O)(C2=CC=C(C)C=C2)=CCN3...,-,5,acrivastine,A166,9874698,30,1,88.0,%,...,%,NaN,NaN,-,NaN,-,NaN,-,NaN,-
4,O(CN1C=2NC(=NC(=O)C2N=C1)N)CCO,-,6,Acyclovir,A166,9874698,30,1,16.0,%,...,%,NaN,NaN,-,NaN,-,NaN,-,NaN,-


## Functions for data preperation

In [12]:
def canonicalize(Dataframe: pd.DataFrame, column_name: str):
    
    """Canonicalizes the SMILES from Dataframe. A column called 'SMILES' is requiered

    Args: 
        Dataframe with 'SMILES' column contaning smiles. 
    """
    
    Dataframe[column_name] = Dataframe[column_name].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x))) #canonicalize smiles from a Dataframe                                          
    

In [14]:
def add_Lipinski_descriptors(Dataframe: pd.DataFrame, column_name: str):

   """This function takes a dataframe and adds the Lipinski descriptors 
      (Molecular weight in Dalton, Number of H-acceptors and donors and the 
      logP value) to it. It also adds a column witch says if the molecule follows 
      the Lipinski's rule of five. (1= yes, 0= no)
    
   Args: 
      Pandas Dataframe with a column witch contains the SMILES strings.
   """
   Dataframe['MolW(Da)'] = Dataframe[column_name].apply(lambda x: Chem.Descriptors.MolWt(Chem.MolFromSmiles(x)))

   Dataframe['NumHAcceptors'] = Dataframe[column_name].apply(lambda x: Chem.Descriptors.NumHAcceptors(Chem.MolFromSmiles(x)))

   Dataframe['NumHDonors'] = Dataframe[column_name].apply(lambda x: Chem.Descriptors.NumHDonors(Chem.MolFromSmiles(x)))

   Dataframe['LogP'] = Dataframe[column_name].apply(lambda x: Chem.Descriptors.MolLogP(Chem.MolFromSmiles(x)))

   Dataframe['Lipinski_rule'] = Dataframe.apply(lambda x: 1 if x['MolW(Da)'] <= 500 and x['NumHAcceptors'] <= 10 and x['NumHDonors'] <= 5 and x['LogP'] <= 5 else 0, axis=1)

In [17]:
def convert_solubility(Dataframe: pd.DataFrame, column_solubility: str, column_Mw: str):
    """
    Converts the solubility values from a DataFrame from log(ug/mL) to log(mol/L).
    
    Args:
        dataframe (pd.DataFrame): DataFrame with the solubility values.
        column_solubility (str): Column name containing solubility values in log(ug/L).
        column_Mw (str): Column name containing molecular weight (Mw) values in g/mol.
    
    Returns:
        pd.DataFrame: DataFrame with an additional column for solubility in log(mol/L).
    """
    # Define a function to apply to each row
    def convert_log_solubility(row):
        log_ug_per_L = row[column_solubility]
        molar_mass = row[column_Mw]
        # Perform the conversion
        log_mol_per_L = log_ug_per_L - 3 - math.log10(molar_mass)
        return log_mol_per_L
    
    # Apply the conversion function to each row and create a new column
    Dataframe['Solubility_log(mol/L)'] = Dataframe.apply(convert_log_solubility, axis=1)
    
    return Dataframe

In [18]:
def calc_Solubility_molL(Dataframe: pd.DataFrame, coulmn_name: str):
    
    """Calculates the solubility in mol/L from a Dataframe with a column containing solubility in log(mol/L).

    Args: 
        Dataframe with column containing solubility values in log(mol/L).
    """
    
    Dataframe['Solubility(mol/L)'] = Dataframe[coulmn_name].apply(lambda x: 10**x)

    return Dataframe

In [19]:
def drop_analysis(Dataframe: pd.DataFrame):

    """Drops duplicates and missing SOL data from a Dataframe.

    Args:
        Dataframe with a column containing SMILES strings and a column containing solubility values.
    """
    
    len = Dataframe.shape[0]
    data = Dataframe.drop_duplicates(subset="SMILES", keep='first')
    data2 = Dataframe.dropna(subset=['Solubility_log(mol/L)'])
    data3 = Dataframe.dropna(subset=['Solubility(mol/L)']).drop_duplicates(subset="SMILES", keep='first')
    print(f"dropped (missing SOL data): {len - data2.shape[0]}\ndropped (duplicates): {len - data.shape[0]}\ndropped (total): {len - data3.shape[0]}")

    return data3

# Preparing data for combining
Here we canonicalize the SMILES strings, add the Lipinski descriptors and add the Solubility in mol/L to the Dataframe. The Water Solubility in all compounds is given as log(mol/L).

In [20]:
canonicalize(df_OCHEM, 'SMILES')
add_Lipinski_descriptors(df_OCHEM, 'SMILES')
calc_Solubility_molL(df_OCHEM, 'Solubility_log(mol/L)')
#saving as csv
df_OCHEM.to_csv('../Data/OCHEM_Sol.csv', index=False)
df_OCHEM.shape #5000

(5000, 13)

In [170]:
canonicalize(df_ESOL, 'SMILES')
add_Lipinski_descriptors(df_ESOL, 'SMILES')
calc_Solubility_molL(df_ESOL, 'Solubility_log(mol/L)')
df_ESOL.to_csv('../Data/ESOL.csv', index=False)
df_ESOL.shape #1128

(1128, 16)

In [172]:
canonicalize(df_biogen, 'SMILES')
add_Lipinski_descriptors(df_biogen, 'SMILES')
calc_Solubility_molL(df_biogen, 'Solubility_log(mol/L)')
df_biogen.to_csv('../Data/Biogen.csv', index=False)
df_biogen.shape #3521

(3521, 17)

In [173]:
canonicalize(df_curatedSol, 'SMILES')
add_Lipinski_descriptors(df_curatedSol, 'SMILES')
calc_Solubility_molL(df_curatedSol, 'Solubility_log(mol/L)')
df_curatedSol.to_csv('../Data/CuratedSol.csv', index=False)
df_curatedSol.shape #9982

[16:59:47] WARNING: not removing hydrogen atom without neighbors
[16:59:47] WARNING: not removing hydrogen atom without neighbors
[16:59:47] WARNING: not removing hydrogen atom without neighbors
[16:59:47] WARNING: not removing hydrogen atom without neighbors
[16:59:47] WARNING: not removing hydrogen atom without neighbors
[16:59:47] WARNING: not removing hydrogen atom without neighbors
[16:59:47] WARNING: not removing hydrogen atom without neighbors
[16:59:47] WARNING: not removing hydrogen atom without neighbors
[16:59:47] WARNING: not removing hydrogen atom without neighbors
[16:59:47] WARNING: not removing hydrogen atom without neighbors
[16:59:47] WARNING: not removing hydrogen atom without neighbors
[16:59:47] WARNING: not removing hydrogen atom without neighbors
[16:59:47] WARNING: not removing hydrogen atom without neighbors
[16:59:47] WARNING: not removing hydrogen atom without neighbors
[16:59:47] WARNING: not removing hydrogen atom without neighbors
[16:59:47] WARNING: not r

(9982, 30)

## Combining data
Combing all the solubility data (ESOL, curated, Biogen, OCHEM) to one big dataframe which contains SMILES, Solubility in log(mol/L), Mw and Lipinski descriptors

In [183]:
df_merged_1 = pd.merge(df_curatedSol, df_biogen, on=['SMILES', 'Solubility_log(mol/L)', 'Solubility(mol/L)', 'MolW(Da)', 'NumHAcceptors', 'NumHDonors', 'LogP', 'Lipinski_rule'], how='outer').filter(['SMILES', 'Solubility_log(mol/L)', 'Solubility(mol/L)', 'MolW(Da)', 'NumHAcceptors', 'NumHDonors', 'LogP', 'Lipinski_rule'])
df_merged_1_cleand = drop_analysis(df_merged_1)

dropped (missing SOL data): 1348
dropped (duplicates): 23
dropped (total): 1356


Here we saw: the biogen datasets has a lot of missing water solubility data.

In [184]:
df_merged_2 = pd.merge(df_merged_1_cleand, df_ESOL, on=['SMILES', 'Solubility_log(mol/L)', 'Solubility(mol/L)', 'MolW(Da)', 'NumHAcceptors', 'NumHDonors', 'LogP', 'Lipinski_rule'], how='outer').filter(['SMILES', 'Solubility_log(mol/L)', 'Solubility(mol/L)', 'MolW(Da)', 'NumHAcceptors', 'NumHDonors', 'LogP', 'Lipinski_rule'])
df_merged_2_cleand = drop_analysis(df_merged_2)

dropped (missing SOL data): 0
dropped (duplicates): 1035
dropped (total): 1035


Here we saw: the ESOL data contained some duplicates

In [185]:
df_merged_3 = pd.merge(df_merged_2_cleand, df_OCHEM, on=['SMILES', 'Solubility_log(mol/L)', 'Solubility(mol/L)', 'MolW(Da)', 'NumHAcceptors', 'NumHDonors', 'LogP', 'Lipinski_rule'], how='outer').filter(['SMILES', 'Solubility_log(mol/L)', 'Solubility(mol/L)', 'MolW(Da)', 'NumHAcceptors', 'NumHDonors', 'LogP', 'Lipinski_rule'])
df_merged_3_cleand = drop_analysis(df_merged_3)

dropped (missing SOL data): 0
dropped (duplicates): 4173
dropped (total): 4173


Here we saw: most of the OCHEM Smiles were already in the other datasets

In [186]:
df_merged_3_cleand.to_csv('../Data/Merged_solubility.csv', index=False)

## Bioavailibility Data

In [ ]:
excel_file_path = 'example.xlsx'

# Read the Excel file into a DataFrame
df = pd.read_excel(excel_file_path)

# Specify the path to the output CSV file
csv_file_path = 'example.csv'

# Write the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

In [23]:
def calc_logK(Dataframe: pd.DataFrame, column_name: str):
    """
    Calculates the logK from a DataFrame with a column containing K values.

    Args:
        Dataframe with column containing %F values.
    """
    Dataframe['logK(%F)'] = Dataframe['%F'].apply(lambda x: -math.log10(x/100-x))

    return Dataframe

In [36]:
df_WEI_Bioav.shape
df_bioavailability.head()

(1157, 7)

In [48]:
df = df_bioavailability.rename(columns={'Updated SMILES': 'SMILES'})
canonicalize(df, 'SMILES')
add_Lipinski_descriptors(df, 'SMILES')
df.to_csv('../Data/Bioavailibility.csv', index=False)

In [52]:
df1 = df_WEI_Bioav.rename(columns={'smile': 'SMILES', 'value': '%F'})
calc_logK(df1, '%F')
canonicalize(df1, 'SMILES')
add_Lipinski_descriptors(df1, 'SMILES')
df1.to_csv('../Data/HOB_Bioavailibility_Wei_2022.csv', index=False)


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [44]:
df_WEI_Bioav.head()
#df_bioavailability.head()

,Name,SMILES,%F,label_cutoff_50%,our model predition_50%,label_cutoff_20%,our model predition_20%
0,sulfadiazine,Nc1ccc(S(=O)(=O)Nc2ncccn2)cc1,90,1,1,1.0,1.0
1,clofarabine,Nc1nc(Cl)nc2c1ncn2C1OC(CO)C(O)C1F,50,1,1,1.0,1.0
2,sulfamethoxazole,Cc1cc(NS(=O)(=O)c2ccc(N)cc2)no1,99,1,1,1.0,1.0
3,tolazoline,c1ccc(CC2=NCCN2)cc1,90,1,1,1.0,1.0
4,cotinine,CN1C(=O)CCC1c1cccnc1,97,1,1,1.0,1.0


In [51]:
canonicalize(df_WEI_Bioav, 'SMILES')

In [54]:
calc_logK(df_WEI_Bioav, '%F')

TypeError: unsupported operand type(s) for /: 'str' and 'int'